In [23]:
import os
from tqdm import tqdm

import utils
import config
import dataset_dfire
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader
import validate

import numpy as np
import torch
import torchmetrics

import cv2
import matplotlib.pyplot as plt

In [24]:
import onnx
import onnxruntime

# Load Models

### No Compression

In [25]:
model_no = onnx.load('./models/onnx/no_comp_model.onnx')
onnx.checker.check_model(model_no)

### Low Compression

In [26]:
model_low = onnx.load('./models/onnx/low_comp_model.onnx')
onnx.checker.check_model(model_low)

### Medium Compression

In [27]:
model_med = onnx.load('./models/onnx/med_comp_model.onnx')
onnx.checker.check_model(model_med)

### High Compression

In [28]:
model_high = onnx.load('./models/onnx/high_comp_model.onnx')
onnx.checker.check_model(model_high)

### FASDD No Compression

In [29]:
model_fasdd_no_comp = onnx.load('./models/onnx/fasdd_no_comp.onnx')
onnx.checker.check_model(model_fasdd_no_comp)

### FASDD Medium Compression

In [30]:
model_fasdd_med = onnx.load('./models/onnx/fasdd_med.onnx')
onnx.checker.check_model(model_fasdd_med)

In [31]:
model_fasdd_med_f1mean = onnx.load('./models/onnx/fasdd_med_best_f1_mean_epoch=37.onnx')
onnx.checker.check_model(model_fasdd_med_f1mean)

### Original FP32 Model Trained with FASDD

In [32]:
model_fp32 = onnx.load('./models/onnx/BED_classifier__fused__dfire_fasdd.onnx')
onnx.checker.check_model(model_fp32)

# Dataset

In [33]:
# VALIDATION DATASET
val_transform = A.Compose([
    A.Resize(config.IMG_H, config.IMG_W, p=1),
    ToTensorV2(p=1),
    ]
)

print("\nTEST DFire dataset")
val_dataset = dataset_dfire.DFireDataset(
    img_h = config.IMG_H,
    img_w = config.IMG_W,
    img_dir = config.VAL_IMG_DIR,
    label_dir = config.VAL_LABEL_DIR,
    num_classes = config.N_CLASSES,
    ds_len = config.DS_LEN,
    transform=val_transform)

print(f'Test dataset len: {len(val_dataset)}')

# LOADERS
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=config.BATCH_SIZE,
                        num_workers=config.NUM_WORKERS,
                        pin_memory=config.PIN_MEMORY,
                        shuffle=False,
                        drop_last=True)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895
Test dataset len: 4306


# Evaluate ONNX with F1 Mean, Smoke and Fire

In [34]:
precision_metric = torchmetrics.classification.MultilabelPrecision(num_labels = config.N_CLASSES, 
                                                                   threshold = 0.5, 
                                                                   average = None).to('cpu')
recall_metric = torchmetrics.classification.MultilabelRecall(num_labels = config.N_CLASSES, 
                                                             threshold = 0.5, 
                                                             average = None).to('cpu')
accuracy_metric = torchmetrics.classification.MultilabelAccuracy(num_labels = config.N_CLASSES, 
                                                                 threshold = 0.5, 
                                                                 average = None).to('cpu')
f1_metric = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
                                                          threshold = 0.5, 
                                                          average = None).to('cpu')

f1_metric_mean = torchmetrics.classification.MultilabelF1Score(num_labels = config.N_CLASSES, 
                                                               threshold = 0.5, 
                                                               average = 'macro').to('cpu')

In [35]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [36]:
def eval_onnx(loader, model_name):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    precision_metric.reset()
    recall_metric.reset()
    accuracy_metric.reset()
    f1_metric.reset()
    f1_metric_mean.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            yhat = ort_session.run(None, ort_inputs)
            yhat = np.array(yhat)
            #yhat = torch.tensor(yhat).squeeze(dim=0)
            yhat = torch.sigmoid(torch.tensor(yhat).squeeze(dim=0))
            target = label[idx].unsqueeze(dim=0)

            precision_metric.update(yhat, target)
            recall_metric.update(yhat, target)
            accuracy_metric.update(yhat, target)
            f1_metric.update(yhat, target)
            f1_metric_mean.update(yhat, target)
    
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    accuracy = accuracy_metric.compute()
    f1 = f1_metric.compute()
    f1_mean = f1_metric_mean.compute()

    precision_metric.reset()
    recall_metric.reset()
    accuracy_metric.reset()
    f1_metric.reset()
    f1_metric_mean.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    print(f'Mean F1 Score: {f1_mean.item():.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()],
        'F1 mean': f1_mean.item(),
        }
    )

In [37]:
_ = eval_onnx(val_loader, './models/onnx/no_comp_model.onnx')

Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:12<00:00,  5.23it/s]

SMOKE -> Precision: 0.9025 - Recall: 0.9021 - Accuracy: 0.9060 - F1: 0.9023
FIRE -> Precision: 0.9352 - Recall: 0.9099 - Accuracy: 0.9604 - F1: 0.9224
Mean F1 Score: 0.9123


In [38]:
_ = eval_onnx(val_loader, './models/onnx/low_comp_model.onnx')

Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:10<00:00,  6.30it/s]

SMOKE -> Precision: 0.9024 - Recall: 0.9011 - Accuracy: 0.9056 - F1: 0.9018
FIRE -> Precision: 0.9216 - Recall: 0.9324 - Accuracy: 0.9620 - F1: 0.9270
Mean F1 Score: 0.9144


In [39]:
_ = eval_onnx(val_loader, './models/onnx/med_comp_model.onnx')

Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:10<00:00,  6.53it/s]

SMOKE -> Precision: 0.9028 - Recall: 0.9001 - Accuracy: 0.9053 - F1: 0.9015
FIRE -> Precision: 0.9346 - Recall: 0.9144 - Accuracy: 0.9613 - F1: 0.9244
Mean F1 Score: 0.9129


In [40]:
_ = eval_onnx(val_loader, './models/onnx/high_comp_model.onnx')

Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:09<00:00,  6.82it/s]

SMOKE -> Precision: 0.9081 - Recall: 0.9001 - Accuracy: 0.9081 - F1: 0.9041
FIRE -> Precision: 0.9416 - Recall: 0.9153 - Accuracy: 0.9634 - F1: 0.9283
Mean F1 Score: 0.9162


# FASDD Models Evaluation

### En este model FP32 tiene que haber algo mal, quizá del onnx
### Evaluar con load & fuse de classifier_modules_names

In [41]:
print("__________________________________ FP32 ORIGINAL MODEL __________________________________")
_ = eval_onnx(val_loader, './models/onnx/BED_classifier__fused__dfire_fasdd.onnx')

__________________________________ FP32 ORIGINAL MODEL __________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:34<00:00,  1.92it/s]

SMOKE -> Precision: 0.9496 - Recall: 0.5846 - Accuracy: 0.7852 - F1: 0.7237
FIRE -> Precision: 0.9629 - Recall: 0.6081 - Accuracy: 0.8925 - F1: 0.7454
Mean F1 Score: 0.7346


In [42]:
print("__________________________________ FASDD NO COMPRESSION __________________________________")
_ = eval_onnx(val_loader, './models/onnx/fasdd_no_comp.onnx')

__________________________________ FASDD NO COMPRESSION __________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:12<00:00,  5.28it/s]

SMOKE -> Precision: 0.9066 - Recall: 0.8982 - Accuracy: 0.9065 - F1: 0.9024
FIRE -> Precision: 0.9163 - Recall: 0.9270 - Accuracy: 0.9592 - F1: 0.9216
Mean F1 Score: 0.9120


In [43]:
print("________________________________ FASDD MEDIUM COMPRESSION ________________________________")
_ = eval_onnx(val_loader, './models/onnx/fasdd_med.onnx')

________________________________ FASDD MEDIUM COMPRESSION ________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:11<00:00,  5.70it/s]

SMOKE -> Precision: 0.9108 - Recall: 0.9011 - Accuracy: 0.9100 - F1: 0.9059
FIRE -> Precision: 0.9221 - Recall: 0.9387 - Accuracy: 0.9636 - F1: 0.9304
Mean F1 Score: 0.9182


In [44]:
print("________________________________ FASDD MEDIUM COMPRESSION ________________________________")
print("________________________________ BEST F1 MEAN EPOCH 37 ________________________________")
_ = eval_onnx(val_loader, './models/onnx/fasdd_med_best_f1_mean_epoch=37.onnx')

________________________________ FASDD MEDIUM COMPRESSION ________________________________
________________________________ BEST F1 MEAN EPOCH 37 ________________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████| 67/67 [00:11<00:00,  5.87it/s]

SMOKE -> Precision: 0.8942 - Recall: 0.9050 - Accuracy: 0.9028 - F1: 0.8995
FIRE -> Precision: 0.9314 - Recall: 0.9297 - Accuracy: 0.9641 - F1: 0.9306
Mean F1 Score: 0.9151
